### Import Essential Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from functools import reduce

##### Extract data from json file

In [12]:
a=pd.read_json (r'./BrahmsConcertPart2/JSON Files/alphapose-results_File18.json')
a.keypoints = a.keypoints.map(lambda x: x[:3])
b=a[['image_id','keypoints']]

##### Filter x,y, and confidence paramters by converting keypoints

In [13]:
b[['x','y','conf']] = pd.DataFrame(b.keypoints.tolist(), index= b.index)
# c=b[(b['conf']>0.5)].reset_index(drop=True)
# c1=c[c['x']<150].reset_index(drop=True)
# c2=c[(c['x']<250)&(c['x']>=150)].reset_index(drop=True)
# c3=c[(c['x']<400)&(c['x']>=250)].reset_index(drop=True)
# c4=c[(c['x']<480)&(c['x']>=400)].reset_index(drop=True)
# c5=c[c['x']>=480].reset_index(drop=True)

C:\Users\San\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


##### This step is used to create movement limits based on range of motion of each participant. Videos should be analyzed prior to setting the limits.

In [1142]:
c1=b[b['x']<165].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)
c2=b[(b['x']<250)&(b['x']>=165)].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)
c3=b[(b['x']<400)&(b['x']>=250)].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)
c4=b[(b['x']<475)&(b['x']>=400)].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)
c5=b[b['x']>=475].sort_values('conf', ascending=False).drop_duplicates(['image_id']).reset_index(drop=True)

##### Test code to check if database has been split properly

In [1148]:
if(((c1.shape[0]+c2.shape[0]+c3.shape[0]+c4.shape[0]+c5.shape[0])/5)==c1.shape[0]):
    print("Perfect")
else:
    print("Error exists")

Perfect


## Get only frame number - to be used as an identifier

In [1149]:
c1.image_id=c1.image_id.str.split(".",1).str[0].astype(int)
c2.image_id=c2.image_id.str.split(".",1).str[0].astype(int)
c3.image_id=c3.image_id.str.split(".",1).str[0].astype(int)
c4.image_id=c4.image_id.str.split(".",1).str[0].astype(int)
c5.image_id=c5.image_id.str.split(".",1).str[0].astype(int)
#c1=c1.sort_values(by="image_id").reset_index(drop=True)

## Code to check if there is any missing sequence

In [1150]:
seq = pd.RangeIndex(c5.image_id.min(), c5.image_id.max())
seq[~seq.isin(c5.image_id)].values

array([], dtype=int64)

In [1151]:
seq = pd.RangeIndex(c4.image_id.min(), c4.image_id.max())
seq[~seq.isin(c4.image_id)].values

array([], dtype=int64)

In [1152]:
seq = pd.RangeIndex(c3.image_id.min(), c3.image_id.max())
seq[~seq.isin(c3.image_id)].values

array([], dtype=int64)

In [1153]:
seq = pd.RangeIndex(c2.image_id.min(), c2.image_id.max())
seq[~seq.isin(c2.image_id)].values

array([], dtype=int64)

In [1154]:
seq = pd.RangeIndex(c1.image_id.min(), c1.image_id.max())
seq[~seq.isin(c1.image_id)].values

array([], dtype=int64)

## Check for duplicates

In [99]:
dataCorrect=c2['image_id'].value_counts().sort_values(ascending=True).reset_index()

In [100]:
dataCorrect[dataCorrect['image_id']>1]

,index,image_id
406,276.jpg,2
407,124.jpg,2
408,419.jpg,2
409,498.jpg,2
410,539.jpg,2
...,...,...
682,372.jpg,2
683,505.jpg,2
684,374.jpg,2
685,457.jpg,3


In [ ]:
### c5 correction

c5[c5['image_id']=='132.jpg']

In [7]:
c5.drop(c5.index[135],inplace=True)
c5.reset_index(drop=True,inplace=True)

## Merge all dataframes

In [1155]:
# cs=[c1,c2,c3,c4,c5]
#cFinal = reduce(lambda left,right: pd.merge(left,right,how="inner",on="image_id"), cs)
cPreFinal=c1.merge(c2.merge(c3.merge(c4.merge(c5,on='image_id',how='inner'),on='image_id',how='inner'),on='image_id',how='inner'),on='image_id',how='inner')

In [1156]:
cPreFinal.columns=['image_id','c1_keypoints','c1_x','c1_y','c1_conf'
                  ,'c2_keypoints','c2_x','c2_y','c2_conf'
                  ,'c3_keypoints','c3_x','c3_y','c3_conf'
                  ,'c4_keypoints','c4_x','c4_y','c4_conf'
                  ,'c5_keypoints','c5_x','c5_y','c5_conf']

In [1157]:
cFinal=cPreFinal[['image_id','c1_x','c1_y','c2_x','c2_y','c3_x','c3_y','c4_x','c4_y','c5_x','c5_y']].sort_values(by='image_id').reset_index(drop=True)

In [1158]:
cFinal

,image_id,c1_x,c1_y,c2_x,c2_y,c3_x,c3_y,c4_x,c4_y,c5_x,c5_y
0,0,128.876801,218.998413,185.380478,226.660141,338.550293,212.332962,466.045654,209.420883,516.034119,211.285706
1,1,128.468674,218.776382,186.290634,225.963898,339.592743,212.493164,465.853119,209.729050,518.018433,210.236526
2,2,129.049973,217.365784,185.029938,226.210953,338.125519,212.463211,465.583618,209.399902,518.097473,210.062531
3,3,127.601044,217.640167,185.626755,224.946579,338.208130,212.372208,465.342102,209.185806,520.535767,209.696609
4,4,127.304443,217.322830,185.284653,224.410934,338.519806,212.584824,465.638306,208.608307,523.760681,207.509140
...,...,...,...,...,...,...,...,...,...,...,...
1114,1114,122.394730,216.391296,192.704956,223.880508,332.349762,209.992355,437.800598,210.685455,536.015442,208.758286
1115,1115,122.735146,216.508728,192.733948,223.215317,332.164459,210.558517,435.551178,210.965927,535.460266,208.933273
1116,1116,122.978729,215.576828,191.667740,224.177826,332.570496,210.547531,435.576233,211.642059,538.127319,208.983109
1117,1117,121.444336,215.859055,191.420456,224.284897,331.941833,210.387711,435.835083,212.441772,537.894348,207.753937


In [1159]:
cFinal['image_id']=np.arange(len(cFinal))

In [1160]:
cFinal

,image_id,c1_x,c1_y,c2_x,c2_y,c3_x,c3_y,c4_x,c4_y,c5_x,c5_y
0,0,128.876801,218.998413,185.380478,226.660141,338.550293,212.332962,466.045654,209.420883,516.034119,211.285706
1,1,128.468674,218.776382,186.290634,225.963898,339.592743,212.493164,465.853119,209.729050,518.018433,210.236526
2,2,129.049973,217.365784,185.029938,226.210953,338.125519,212.463211,465.583618,209.399902,518.097473,210.062531
3,3,127.601044,217.640167,185.626755,224.946579,338.208130,212.372208,465.342102,209.185806,520.535767,209.696609
4,4,127.304443,217.322830,185.284653,224.410934,338.519806,212.584824,465.638306,208.608307,523.760681,207.509140
...,...,...,...,...,...,...,...,...,...,...,...
1114,1114,122.394730,216.391296,192.704956,223.880508,332.349762,209.992355,437.800598,210.685455,536.015442,208.758286
1115,1115,122.735146,216.508728,192.733948,223.215317,332.164459,210.558517,435.551178,210.965927,535.460266,208.933273
1116,1116,122.978729,215.576828,191.667740,224.177826,332.570496,210.547531,435.576233,211.642059,538.127319,208.983109
1117,1117,121.444336,215.859055,191.420456,224.284897,331.941833,210.387711,435.835083,212.441772,537.894348,207.753937


## Compute Time, Distance, Velocity, and Acceleration 

### Time

In [1161]:
t_interval=0.0333333333333333

In [1162]:
for i,rows in cFinal.iterrows():
    if(i==0):
        cFinal.loc[i, 't'] = 0
    else:
        cFinal.loc[i,'t'] = cFinal.loc[i-1,'t'] + t_interval;

### Distance, Velocity, and Acceleration

In [1163]:
def va(a,b,c):
    hello=a.copy()
    cold= str(a[b].name[0:2])+'_d'
    colv= str(a[b].name[0:2])+'_v'
    cola= str(a[b].name[0:2])+'_a'
    for i,rows in a.iterrows():
        if(i==0):
            hello.loc[i, cold] = 0
            hello.loc[i, colv] = 0
            hello.loc[i, cola] = 0
        else:
            hello.loc[i, cold] = math.sqrt((hello.loc[i,str(a[b].name[0:2])+'_x'] - hello.loc[i-1,str(a[b].name[0:2])+'_x'])**2 + (hello.loc[i,str(a[b].name[0:2])+'_y'] - hello.loc[i-1,str(a[b].name[0:2])+'_y'])**2)
            hello.loc[i,'t'] = hello.loc[i-1,'t'] + t_interval
            hello.loc[i, colv] = abs((hello.loc[i,cold] - hello.loc[i-1,cold]))/(hello.loc[i,'t'] - hello.loc[i-1,'t'])
            hello.loc[i, cola] = (hello.loc[i, colv] - hello.loc[i-1,colv])/(hello.loc[i,'t'] - hello.loc[i-1,'t'])
    return hello

In [1164]:
cFinal2=va(cFinal,'c1_x','c1_y')
cFinal3=va(cFinal2,'c2_x','c2_y')
cFinal4=va(cFinal3,'c3_x','c3_y')
cFinal5=va(cFinal4,'c4_x','c4_y')
d=va(cFinal5,'c5_x','c5_y')

In [1165]:
d

,image_id,c1_x,c1_y,c2_x,c2_y,c3_x,c3_y,c4_x,c4_y,c5_x,...,c2_a,c3_d,c3_v,c3_a,c4_d,c4_v,c4_a,c5_d,c5_v,c5_a
0,0,128.876801,218.998413,185.380478,226.660141,338.550293,212.332962,466.045654,209.420883,516.034119,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,128.468674,218.776382,186.290634,225.963898,339.592743,212.493164,465.853119,209.729050,518.018433,...,1031.330539,1.054688,31.640637,949.219120,0.363368,10.901047,327.031412,2.244611,67.338331,2020.149921
2,2,129.049973,217.365784,185.029938,226.210953,338.125519,212.463211,465.583618,209.399902,518.097473,...,-906.452954,1.467530,12.385258,-577.661392,0.425404,1.861075,-271.199169,0.191107,61.605111,-171.996577
3,3,127.601044,217.640167,185.626755,224.946579,338.208130,212.372208,465.342102,209.185806,520.535767,...,-22.748078,0.122907,40.338675,838.602518,0.322749,3.079641,36.556973,2.465598,68.234719,198.888220
4,4,127.304443,217.322830,185.284653,224.410934,338.519806,212.584824,465.638306,208.608307,523.760681,...,584.195372,0.377290,7.631470,-981.216146,0.649032,9.788469,201.264859,3.896806,42.936240,-758.954352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,1114,122.394730,216.391296,192.704956,223.880508,332.349762,209.992355,437.800598,210.685455,536.015442,...,230.511761,0.737148,12.341353,285.851003,0.887339,14.145492,14.156128,3.051855,34.749935,928.947331
1115,1115,122.735146,216.508728,192.733948,223.215317,332.164459,210.558517,435.551178,210.965927,535.460266,...,-193.999782,0.595715,4.242973,-242.951404,2.266838,41.384989,817.184908,0.582100,74.092644,1180.281266
1116,1116,122.978729,215.576828,191.667740,224.177826,332.570496,210.547531,435.576233,211.642059,538.127319,...,403.982617,0.406185,5.685908,43.288036,0.676596,47.707259,189.668089,2.667519,62.562551,-345.902776
1117,1117,121.444336,215.859055,191.420456,224.284897,331.941833,210.387711,435.835083,212.441772,537.894348,...,356.718748,0.648659,7.274224,47.649502,0.840562,4.918972,-1283.648600,1.251055,42.493911,-602.059198


In [1166]:
d['t'].max()

37.266666666667106

# In order to make changes to the total number of rows a multiple of 15 & 3. Very important step.

## To trim the dataframe to make it a multiple of 15 & 3

In [1167]:
d=d[0:1110]

In [1168]:
d

,image_id,c1_x,c1_y,c2_x,c2_y,c3_x,c3_y,c4_x,c4_y,c5_x,...,c2_a,c3_d,c3_v,c3_a,c4_d,c4_v,c4_a,c5_d,c5_v,c5_a
0,0,128.876801,218.998413,185.380478,226.660141,338.550293,212.332962,466.045654,209.420883,516.034119,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,128.468674,218.776382,186.290634,225.963898,339.592743,212.493164,465.853119,209.729050,518.018433,...,1031.330539,1.054688,31.640637,949.219120,0.363368,10.901047,327.031412,2.244611,67.338331,2020.149921
2,2,129.049973,217.365784,185.029938,226.210953,338.125519,212.463211,465.583618,209.399902,518.097473,...,-906.452954,1.467530,12.385258,-577.661392,0.425404,1.861075,-271.199169,0.191107,61.605111,-171.996577
3,3,127.601044,217.640167,185.626755,224.946579,338.208130,212.372208,465.342102,209.185806,520.535767,...,-22.748078,0.122907,40.338675,838.602518,0.322749,3.079641,36.556973,2.465598,68.234719,198.888220
4,4,127.304443,217.322830,185.284653,224.410934,338.519806,212.584824,465.638306,208.608307,523.760681,...,584.195372,0.377290,7.631470,-981.216146,0.649032,9.788469,201.264859,3.896806,42.936240,-758.954352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,1105,122.627800,217.114883,193.368591,224.270798,330.816864,209.910690,436.664093,210.523239,528.181641,...,828.917072,0.715818,11.001641,-332.676632,1.230817,21.956513,533.181992,1.542734,29.132082,-510.909978
1106,1106,122.372322,217.294754,193.331375,223.745926,331.173157,209.904236,436.943787,209.112076,528.137207,...,-1029.109651,0.356351,10.783999,-6.529246,1.438614,6.233914,-471.677965,0.070238,44.174876,451.283816
1107,1107,122.516327,216.529388,192.133469,223.636185,331.844971,210.009384,437.276978,210.600174,529.480408,...,589.496866,0.679993,9.709248,-32.242552,1.524943,2.589879,-109.321065,1.932213,55.859223,350.530393
1108,1108,122.367157,216.529526,192.702393,224.487564,331.468079,209.525558,436.265869,209.964569,529.767578,...,-448.005351,0.613298,2.000831,-231.252504,1.194292,9.919537,219.889736,0.704274,36.838170,-570.631590


In [1169]:
d['t'].max()

36.96666666666712

## To add/replicate rows to balance as a multiple of 15 & 3

In [1035]:
d=d.append([d.iloc[717]]*2).reset_index(drop=True)

In [1036]:
d['image_id']=np.arange(len(d))

In [1037]:
d['t']

0       0.000000
1       0.033333
2       0.066667
3       0.100000
4       0.133333
         ...    
715    23.833333
716    23.866667
717    23.900000
718    23.900000
719    23.900000
Name: t, Length: 720, dtype: float64

In [1038]:
for i,rows in d.iterrows():
    if(i==0):
        d.loc[i, 't'] = 0
    else:
        d.loc[i,'t'] = d.loc[i-1,'t'] + t_interval;

In [1039]:
d

,image_id,c1_x,c1_y,c2_x,c2_y,c3_x,c3_y,c4_x,c4_y,c5_x,...,c2_a,c3_d,c3_v,c3_a,c4_d,c4_v,c4_a,c5_d,c5_v,c5_a
0,0,100.991219,215.978195,187.368973,216.598068,345.586090,212.520737,454.131958,216.420914,529.327087,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,99.858200,216.097275,187.485809,217.383286,344.418457,212.306473,453.733215,216.240005,531.322571,...,714.475850,1.187129,35.613879,1068.416372,0.437862,13.135873,394.076179,2.133263,63.997896,1919.936881
2,2,101.175278,215.050186,187.791367,218.605988,344.663513,212.557571,452.824707,216.759781,535.353455,...,-294.678348,0.350860,25.088066,-315.774387,1.046687,18.264739,153.865991,4.031512,56.947463,-211.512978
3,3,100.047226,214.778519,187.832489,217.766647,344.349609,211.799576,453.249695,217.346817,537.523865,...,-41.836430,0.820422,14.086860,-330.036199,0.724725,9.658867,-258.176147,3.127746,27.112966,-895.034923
4,4,98.943626,214.732681,187.243118,218.425308,344.364441,212.161285,452.835938,217.085083,538.888306,...,-338.807742,0.362014,13.752266,-10.037802,0.489591,7.054000,-78.146026,1.365522,52.866721,772.612646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,715,109.942390,214.680069,191.433334,219.942719,329.651276,213.317261,454.415863,216.806000,546.833923,...,1470.519456,1.403382,0.292930,-676.013959,1.712543,20.563739,307.305236,3.428223,71.483235,749.685157
716,716,108.852234,213.355682,193.964432,221.826279,330.166382,213.496658,454.711823,217.402847,547.561523,...,183.163823,0.545452,25.737910,763.349414,0.666197,31.390358,324.798573,0.909464,75.562789,122.386599
717,717,109.101471,214.537109,194.670059,222.198532,330.093567,213.610596,454.686676,217.296524,547.999084,...,217.449359,0.135217,12.307039,-402.926144,0.109256,16.708228,-440.463913,0.691127,6.550108,-2070.380409
718,718,109.101471,214.537109,194.670059,222.198532,330.093567,213.610596,454.686676,217.296524,547.999084,...,217.449359,0.135217,12.307039,-402.926144,0.109256,16.708228,-440.463913,0.691127,6.550108,-2070.380409


## Create DB

In [1170]:
d.to_csv('./BrahmsConcertPart2/Keypoint Data/BrahmsConcertPart2_File18.csv')